In [2]:
!pip install -q datasets transformers[torch] accelerate evaluate shap sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("iNeil77/CodeNet", "C++")
print(ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

C++/train-00000-of-00024.parquet:   0%|          | 0.00/97.7M [00:00<?, ?B/s]

C++/train-00001-of-00024.parquet:   0%|          | 0.00/160M [00:00<?, ?B/s]

...

C++/train-00022-of-00024.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

C++/train-00023-of-00024.parquet:   0%|          | 0.00/137M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7982624 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['s_id', 'p_id', 'u_id', 'date', 'language', 'original_language', 'filename_ext', 'status', 'cpu_time', 'memory', 'code_size', 'code'],
        num_rows: 7982624
    })
})


In [5]:

from collections import Counter
import random

# Add bug labels
def add_labels(example):
    example["bug_label"] = 0 if example["status"] == "Accepted" else 1
    return example

ds["train"] = ds["train"].map(add_labels)

# Check label distribution before sampling
print("Original bug label distribution:", Counter(ds["train"]["bug_label"]))

# Shuffle dataset for randomness
ds["train"] = ds["train"].shuffle(seed=42)

# Keep only 200,000 examples total
target_size = 15_000

labels = ds["train"]["bug_label"]
indices_0 = [i for i, l in enumerate(labels) if l == 0][:target_size // 2]
indices_1 = [i for i, l in enumerate(labels) if l == 1][:target_size // 2]
selected_indices = indices_0 + indices_1
random.shuffle(selected_indices)

ds_small = ds["train"].select(selected_indices)

# Check new label distribution
print("Sampled dataset size:", len(ds_small))
print("Sampled bug label distribution:", Counter(ds_small["bug_label"]))

# Optional: replace the original train split with the smaller one
ds["train"] = ds_small



Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Original bug label distribution: Counter({1: 7500, 0: 7500})
Sampled dataset size: 15000
Sampled bug label distribution: Counter({1: 7500, 0: 7500})


In [6]:
from transformers import AutoTokenizer

MODEL_NAME = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN = 256  # max tokens per code snippet

def preprocess_batch(examples):
    codes = [c if isinstance(c, str) else "" for c in examples["code"]]
    toks = tokenizer(codes, padding="max_length", truncation=True, max_length=MAX_LEN)
    toks["labels"] = examples["bug_label"]
    return toks

tokenized = ds["train"].map(preprocess_batch, batched=True, remove_columns=ds["train"].column_names)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
print(tokenized)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15000
})


In [7]:
from torch.utils.data import DataLoader

# Use all for training; small slice for quick evaluation
train_dataset = tokenized
eval_dataset = train_dataset.select(range(min(200, len(train_dataset))))

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)


In [8]:
from transformers import AutoModelForSequenceClassification
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./codebert-bug-mvp",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=400,
    save_total_limit=1,
    do_eval=True,
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1566816063.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Step,Training Loss


In [ ]:
trainer.save_model("./codebert-bug-mvp-model")
tokenizer.save_pretrained("./codebert-bug-mvp-model")
